In [41]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.impute import KNNImputer

df = pd.read_csv('../train.csv')
df = df.drop(columns=['Name', 'PassengerId'])

binary_features = ['VIP', 'CryoSleep']
cryo_sleep_depending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features_without_cabin = ['HomePlanet', 'Destination']
categorical_features_cleared = ['HomePlanet', 'Destination', 'Deck', 'Side']
numerical_features = ['ShoppingMall', 'Spa', 'VRDeck', 'RoomService', 'FoodCourt', 'Age']
imputer = KNNImputer(n_neighbors=1)

for i in range(len(df)):
    if df.at[i, 'CryoSleep'] == True:
        for feature in cryo_sleep_depending_features:
            df.at[i, feature] = 0.0

rows_with_many_missing_values = df[df.isna().sum(axis=1) > 1]
rows_with_one_missing_value = df[df.isna().sum(axis=1) == 1]
df = df.dropna()     
df[["Deck", "CabinNumber", "Side"]] = df["Cabin"].str.split("/", expand=True)
df = df.drop(columns=['Cabin', 'CabinNumber'])
df = df.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
df = pd.get_dummies(df, columns=categorical_features_cleared)

for feature in numerical_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in categorical_features_without_cabin:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in binary_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value['Cabin'].isna()] 
rows_with_missing_feature = rows_with_missing_feature.drop(columns='Cabin')
rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_without_cabin)
df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_many_missing_values = rows_with_many_missing_values.drop(columns='Cabin')

df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

target = 'Transported'
X = df.drop(columns=target)
y = df[target]

model = CatBoostClassifier(iterations=247, learning_rate=0.05322212136382149, depth=5, random_state=42)
model.fit(X, y)

C:\Users\vitya\AppData\Local\Temp\ipykernel_14920\1429671478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ipykernel_14920\1429671478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ip

0:	learn: 0.6688290	total: 2.11ms	remaining: 520ms
1:	learn: 0.6452640	total: 8.52ms	remaining: 1.04s
2:	learn: 0.6249341	total: 10.5ms	remaining: 857ms
3:	learn: 0.6065274	total: 12.6ms	remaining: 764ms
4:	learn: 0.5911436	total: 14.4ms	remaining: 698ms
5:	learn: 0.5779923	total: 16.4ms	remaining: 657ms
6:	learn: 0.5656548	total: 24ms	remaining: 824ms
7:	learn: 0.5556381	total: 26.3ms	remaining: 785ms
8:	learn: 0.5477497	total: 28.3ms	remaining: 749ms
9:	learn: 0.5404784	total: 30.3ms	remaining: 718ms
10:	learn: 0.5346613	total: 32.1ms	remaining: 690ms
11:	learn: 0.5275620	total: 39.9ms	remaining: 781ms
12:	learn: 0.5197070	total: 41.9ms	remaining: 755ms
13:	learn: 0.5139444	total: 44ms	remaining: 732ms
14:	learn: 0.5090591	total: 46ms	remaining: 712ms
15:	learn: 0.5038940	total: 48ms	remaining: 693ms
16:	learn: 0.5003952	total: 55.3ms	remaining: 748ms
17:	learn: 0.4958984	total: 57.2ms	remaining: 728ms
18:	learn: 0.4911767	total: 59.2ms	remaining: 711ms
19:	learn: 0.4864117	total: 61

In [42]:
df = pd.read_csv('../test.csv')
submission_id = df.PassengerId
df = df.drop(columns=['Name'])

for i in range(len(df)):
    if df.at[i, 'CryoSleep'] == True:
        for feature in cryo_sleep_depending_features:
            df.at[i, feature] = 0.0

rows_with_many_missing_values = df[df.isna().sum(axis=1) > 1]
rows_with_one_missing_value = df[df.isna().sum(axis=1) == 1]
df = df.dropna()     
df[["Deck", "CabinNumber", "Side"]] = df["Cabin"].str.split("/", expand=True)
df = df.drop(columns=['Cabin', 'CabinNumber'])
df = df.astype({'CryoSleep': int, 'VIP': int})
df = pd.get_dummies(df, columns=categorical_features_cleared)

for feature in numerical_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    ids = df['PassengerId']
    df = df.drop(columns=['PassengerId'])
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    df['PassengerId'] = ids

for feature in categorical_features_without_cabin:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    ids = df['PassengerId']
    df = df.drop(columns=['PassengerId'])
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    df['PassengerId'] = ids

for feature in binary_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    ids = df['PassengerId']
    df = df.drop(columns=['PassengerId'])
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    df['PassengerId'] = ids

rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value['Cabin'].isna()] 
rows_with_missing_feature = rows_with_missing_feature.drop(columns='Cabin')
rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int})
rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_without_cabin)
df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
ids = df['PassengerId']
df = df.drop(columns=['PassengerId'])
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df['PassengerId'] = ids

rows_with_many_missing_values = rows_with_many_missing_values.drop(columns=['Cabin', 'HomePlanet', 'Destination'])

df = pd.concat([df, rows_with_many_missing_values], ignore_index=True).reset_index(drop=True)
ids = df['PassengerId']
df = df.drop(columns=['PassengerId'])
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# df['PassengerId'] = ids

predictions = model.predict(df).astype(bool)
output = pd.DataFrame({'PassengerId': ids,'Transported': predictions})
output.to_csv('../output.csv', index=False)

C:\Users\vitya\AppData\Local\Temp\ipykernel_14920\1499703766.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ipykernel_14920\1499703766.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ip